In [54]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time
import configparser

In [82]:
# Read the config file
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [56]:
def auth():
    returnurn config['twitter']['bearer_token']

In [57]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [58]:
def create_url(keyword, start_date, end_date, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent"

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'text',
                    'next_token': {}}
    return (search_url, query_params)

In [59]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token 
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [69]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "bitcoin lang:en"
start_time = "2023-01-17T00:00:00.000Z"
end_time = "2023-01-18T00:00:00.000Z"
max_results = 100

In [70]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [71]:
# print(json.dumps(json_response, indent=4, sort_keys=True))

json_response['data'][0]['text']

'RT @HKD_exchange: #BTC Retakes $20K, Leading as Broad Crypto Rally Continues\n\n🚀 Now at $20,900, #bitcoin has gained more than 20% in the op…'

In [75]:
df = pd.DataFrame(json_response['data'])
df['text']

0     RT @HKD_exchange: #BTC Retakes $20K, Leading a...
1     @hxn @AP Sure, make us crash faster. https://t...
2     RT @adaswapapp: Hey AdaSwap! We are opening a ...
3     RT @Diditaihuttu: This is very important as sc...
4     @MonstersCoins Bear markets provide the greate...
                            ...                        
95    RT @saylor: Join us Wednesday, January 18 at 5...
96    RT @Cointelegraph: Looking to get involved wit...
97    RT @BTC_Archive: #Bitcoin 2-week MACD just cro...
98    This is what most people do when they know the...
99    @nypost Those who believe in investing in Bitc...
Name: text, Length: 100, dtype: object

In [78]:
all_text_for_tweets = ' '.join(df['text'].tolist())
all_text_for_tweets

'RT @HKD_exchange: #BTC Retakes $20K, Leading as Broad Crypto Rally Continues\n\n🚀 Now at $20,900, #bitcoin has gained more than 20% in the op… @hxn @AP Sure, make us crash faster. https://t.co/17VFF01LaH RT @adaswapapp: Hey AdaSwap! We are opening a $2000 prize for anyone who ads LP to our DEX via the Liquidity pools! If you add over $20 USD… RT @Diditaihuttu: This is very important as scammers are using our name daily to scam people in  #bitcoin. Imho we have enough internationa… @MonstersCoins Bear markets provide the greatest accumulating opportunity for those with conviction in #bitcoin, but they are the most dangerous for speculators. \n\nIf you aren’t following @JeremyD_Trades you should be. There aren’t many who have a better understanding of value investing @observadorpt Those who believe in investing in Bitcoin also believe in intelligence Sometimes we only have to take risk once it pays off Thank you Ms. Natalie Taylor , I made the profit of $68,000 ...join her official tele

In [79]:
# feed the text to ibm api and get sentiment score

In [80]:
pip install --upgrade "ibm-watson>=6.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 2.1 MB/s eta 0:00:00
  Created wheel for ibm-watson: filename=ibm_watson-6.1.0-py3-none-any.whl size=370748 sha256=7bd9b0a99f968d9ed564e1359ee332cc391792866e4a2b54f8bebda642221f90
  Stored in directory: /Users/abdullah/Library/Caches/pip/wheels/34/b4/cd/829a351c802b7a578115fe7ddaedff62b29eae84e90882c7e2
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.16.1-py3-none-any.whl size=83552 sha256=9c1c2c5443dfedd7df2cbd7caa35df79431b93cc4e9e837c21437582854fa465
  Stored in directory: /U

In [81]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

In [83]:
# config['twitter']['ibm_api_key']
authenticator = IAMAuthenticator(config['twitter']['ibm_api_key'])
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator)

natural_language_understanding.set_service_url(config['twitter']['ibm_url'])

In [85]:
response = natural_language_understanding.analyze(
    text=all_text_for_tweets,
    features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True, limit=2),
        keywords=KeywordsOptions(emotion=True, sentiment=True,
                                 limit=2))).get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 2,
    "text_characters": 15015,
    "features": 2
  },
  "language": "en",
  "keywords": [
    {
      "text": "RT @HKD_exchange",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "relevance": 0.832045,
      "emotion": {
        "sadness": 0.146121,
        "joy": 0.139064,
        "fear": 0.035868,
        "disgust": 0.087992,
        "anger": 0.091872
      },
      "count": 2
    },
    {
      "text": "RT @adaswapapp",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "relevance": 0.684192,
      "emotion": {
        "sadness": 0.081174,
        "joy": 0.77509,
        "fear": 0.045336,
        "disgust": 0.144504,
        "anger": 0.030906
      },
      "count": 1
    }
  ],
  "entities": [
    {
      "type": "Hashtag",
      "text": "#Bitcoin",
      "sentiment": {
        "score": -0.450832,
        "mixed": "1",
        "label": "negative"
      },
      "relevance": 0.951034